# Build a variational autoencoder

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [2]:
n_train = 10000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg()[-3:] for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
test_strs = [g.get_reg()[-3:] for _ in range(n_test)]
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print(vec_length)

108


### Variational autoencoder

In [3]:
latent_dim = 50
intermediate_dim = 200
original_dim = vec_length

inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [4]:
vae.fit(train_vecs, train_vecs,
        epochs=50,
        batch_size=256,
        shuffle=True,
        validation_data=(test_vecs, test_vecs))

Epoch 1/50
40/40 [==============================] - 1s 25ms/step - loss: 67.2349 - val_loss: 56.8937
Epoch 2/50
40/40 [==============================] - 1s 18ms/step - loss: 51.1828 - val_loss: 46.6528
Epoch 3/50
40/40 [==============================] - 1s 17ms/step - loss: 44.0286 - val_loss: 41.6228
Epoch 4/50
40/40 [==============================] - 1s 17ms/step - loss: 40.0241 - val_loss: 38.4050
Epoch 5/50
40/40 [==============================] - 1s 19ms/step - loss: 37.3326 - val_loss: 36.1003
Epoch 6/50
40/40 [==============================] - 1s 21ms/step - loss: 35.2510 - val_loss: 34.2908
Epoch 7/50
40/40 [==============================] - 1s 24ms/step - loss: 33.6042 - val_loss: 32.7571
Epoch 8/50
40/40 [==============================] - 1s 19ms/step - loss: 32.2675 - val_loss: 31.5135
Epoch 9/50
40/40 [==============================] - 1s 19ms/step - loss: 31.1152 - val_loss: 30.5316
Epoch 10/50
40/40 [==============================] - 1s 19ms/step - loss: 30.1915 - val_los

In [5]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['FVI', 'DCR', 'JWO', 'XKI', 'FNK', 'XIM', 'NON', 'JCE', 'KTZ', 'DFU']
['FVI', 'DCR', 'JWO', 'XKI', 'VNK', 'XIM', 'NON', 'VCE', 'KTZ', 'DFU']
accuracy = 0.87
